In [23]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

In [ ]:
# api_end = "https://api.scryfall.com" # using tapped out api instaed. 
mtg_whole = pd.read_csv("mtg_modern_whole.csv")

mtg_whole.dtypes

In [ ]:
name_, id_= [],[] # this code pulls unique names and id's of cards for removing reprints.
for i,el in enumerate(mtg_whole['name']):
    if el not in name_:
        name_.append(el)
        id_.append(mtg_whole['id'][i])
len(id_)

In [ ]:
# removes reprints from data
mtg_whole = mtg_whole.query(f"id == {id_}")
# writes the cleanned dataframe to a csv. .(always save work!!!)
mtg_whole.to_csv('mtg_modern_clean.csv')

In [3]:
mtg_modern = pd.read_csv('mtg_modern_clean.csv')

/home/ed/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (9,15,39) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
mtg_modern['name'].where(mtg_modern['text'].isna()).dropna(how='all').shape # cards with no rules text. 

(314,)

In [5]:
mtg_modern.head(10)

,Unnamed: 0,Unnamed: 0.1,id,artist,borderColor,colorIdentity,colorIndicator,colors,convertedManaCost,duelDeck,edhrecRank,faceConvertedManaCost,flavorText,frameEffect,frameEffects,frameVersion,hand,hasFoil,hasNoDeckLimit,hasNonFoil,isAlternative,isArena,isBuyABox,isDateStamped,isFullArt,isMtgo,isOnlineOnly,isOversized,isPaper,isPromo,isReprint,isReserved,isStarter,isStorySpotlight,isTextless,isTimeshifted,layout,leadershipSkills,life,loyalty,manaCost,mcmId,mcmMetaId,mtgArenaId,mtgoFoilId,mtgoId,multiverseId,name,names,number,originalText,originalType,otherFaceIds,power,printings,purchaseUrls,rarity,scryfallId,scryfallIllustrationId,scryfallOracleId,setCode,side,subtypes,supertypes,tcgplayerProductId,text,toughness,type,types,uuid,variations,watermark
0,0,0,1,Rebecca Guay,black,G,NaN,G,4.0,NaN,980.0,NaN,NaN,NaN,NaN,2003,NaN,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,normal,NaN,NaN,NaN,{2}{G}{G},16413.0,19.0,NaN,27283.0,27282.0,130483.0,Abundance,NaN,249,"If you would draw a card, you may instead choo...",Enchantment,NaN,NaN,"10E,C17,DDR,USG",{'cardmarket': 'https://mtgjson.com/links/2b93...,rare,46184f97-d5c9-4a98-9fd9-e19057ce9b7e,4be12219-df08-4a40-b32f-69737db62615,eccc9a54-2b56-4a02-9926-258d5b2e25fb,10E,NaN,NaN,NaN,15023.0,"If you would draw a card, you may instead choo...",NaN,Enchantment,Enchantment,1669af17-d287-5094-b005-4b143441442f,NaN,NaN
1,1,1,2,Stephen Daniele,black,U,NaN,U,3.0,NaN,10548.0,NaN,They brandish their latest theories as warrior...,NaN,NaN,2003,NaN,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,normal,NaN,NaN,NaN,{1}{U}{U},16227.0,26.0,NaN,27275.0,27274.0,132072.0,Academy Researchers,NaN,63,"When Academy Researchers comes into play, you ...",Creature - Human Wizard,NaN,2,"10E,USG",{'cardmarket': 'https://mtgjson.com/links/27c7...,uncommon,b62ca800-ec24-476e-b56f-834eef9622a9,cf5efc65-9aa2-466b-bd2b-448ad69ab474,df2d2c52-475b-4057-9cf8-7886700f0fae,10E,NaN,"Human,Wizard",NaN,15024.0,When Academy Researchers enters the battlefiel...,2,Creature — Human Wizard,Creature,047d5499-a21c-5f5c-9679-1599fcaf9815,NaN,NaN
2,2,2,3,John Avon,black,"U,W",NaN,NaN,0.0,NaN,570.0,NaN,NaN,NaN,NaN,2003,NaN,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,normal,NaN,NaN,NaN,NaN,16511.0,42.0,NaN,27385.0,27384.0,129458.0,Adarkar Wastes,NaN,347,{T}: Add {1} to your mana pool.\n{T}: Add {W} ...,Land,NaN,NaN,"10E,5ED,6ED,7ED,9ED,ICE,PTC,WC00",{'cardmarket': 'https://mtgjson.com/links/7866...,rare,f31cf014-7ac9-428b-9ce9-8ba5ebfdd252,1b3845b2-2bac-4cfe-a0c5-6e9f4bd1edb8,d5ad26cc-2bdb-46b7-b8bf-dd099d5fa09b,10E,NaN,NaN,NaN,15025.0,{T}: Add {C}.\n{T}: Add {W} or {U}. Adarkar Wa...,NaN,Land,Land,ee19938c-4007-58f1-8904-fae28007b422,NaN,NaN
3,3,3,4,Roger Raupp,black,B,NaN,B,3.0,NaN,15758.0,NaN,"One rarely notices a heartbeat, save when it i...",NaN,NaN,2003,NaN,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,normal,NaN,NaN,NaN,{2}{B},16289.0,66.0,NaN,27483.0,27482.0,135206.0,Afflict,NaN,125,Target creature gets -1/-1 until end of turn.\...,Instant,NaN,NaN,"10E,ODY",{'cardmarket': 'https://mtgjson.com/links/02e8...,common,20f18447-2853-469b-b3b3-20d6bcafa6e7,83259a38-bbad-4dd1-bcd5-d1b1d96b0380,471f6ad3-cf01-42dd-a519-ea8e13039fea,10E,NaN,NaN,NaN,15026.0,Target creature gets -1/-1 until end of turn.\...,NaN,Instant,Instant,8774e18f-3752-5c06-af94-5da3960da9ed,NaN,NaN
4,4,4,5,Christopher Moeller,black,G,NaN,G,2.0,NaN,9886.0,NaN,"The power of the wild, concentrated in a singl...",NaN,NaN,2003,NaN,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,normal,NaN,NaN,NaN,{1}{G},16414.0,76.0,NaN,27361.0,27360.0,130525.0,Aggressive Urge,NaN,250,Target creature gets +1/+1 until end of turn.\...,Instant,NaN,NaN,"10E,INV,MB1,RIX",{'cardmarket': 'https://mtgjson.com/links/2ec1...,common,bf92ae01-9d1e-4b41-b068-096648daadf6,f4a2d3c4-15b8-4808-a5c5-56aaefc0dfcc,43f5a93b-0f8d-48d2-ab9d-275d44cf88b5,10E,NaN,NaN,NaN,15027.0,Target creature gets +1/+1 until end of turn.\...,NaN,Instant,Instant,4e875bca-0c52-5d60-889d-1db67e261737,NaN,NaN
5,5,5,6,Adam Rex,black,B,NaN,B,4.0,NaN,16665.0,NaN,"In the aftermat

In [6]:
# pulls only usable data for clustering.
mtg_whole_2 = mtg_modern[['name', 'names', 'power', 'toughness', 'text', 'type', 'types', 'subtypes', 'supertypes', 'rarity', 'colorIdentity', 'colors', 'convertedManaCost', 'manaCost', 'faceConvertedManaCost',  'side', ]] # filters that would matter.
#'id', 'borderColor', 'enhrecRank', 'tcgplayerProductId', 'uuid', 'watermark', 'variations', 'originalText', 'originalType', 'scryfallIllustrationId', 'scryfallOracleId'])
mtg_whole_2.head(3)

,name,names,power,toughness,text,type,types,subtypes,supertypes,rarity,colorIdentity,colors,convertedManaCost,manaCost,faceConvertedManaCost,side
0,Abundance,NaN,NaN,NaN,"If you would draw a card, you may instead choo...",Enchantment,Enchantment,NaN,NaN,rare,G,G,4.0,{2}{G}{G},NaN,NaN
1,Academy Researchers,NaN,2,2,When Academy Researchers enters the battlefiel...,Creature — Human Wizard,Creature,"Human,Wizard",NaN,uncommon,U,U,3.0,{1}{U}{U},NaN,NaN
2,Adarkar Wastes,NaN,NaN,NaN,{T}: Add {C}.\n{T}: Add {W} or {U}. Adarkar Wa...,Land,Land,NaN,NaN,rare,"U,W",NaN,0.0,NaN,NaN,NaN


In [7]:
names = ['Green','Blue','Black','Red','White']
start_ = ['G','U','B','R','W']

for i,el in enumerate(names): # creates columns with cost of colors
    mtg_whole_2[f'manaCost_{el}'] = mtg_whole_2['manaCost'].apply(lambda x: str(x).count(f'{start_[i]}'))
# bellow gennerates cost of colorless. 
mtg_whole_2['manaCost_colorless'] = mtg_whole_2['convertedManaCost'].astype('int') - sum([mtg_whole_2['manaCost_Green'],mtg_whole_2['manaCost_Red'],mtg_whole_2['manaCost_Black'],mtg_whole_2['manaCost_Blue'], mtg_whole_2['manaCost_White']])

for i,el in enumerate(names):
    # identityVV
    mtg_whole_2[f'colorIdentity_{el}'] = mtg_whole_2['colorIdentity'].apply(lambda x: str(x).count(f'{start_[i]}'))
mtg_whole_2['colorIdentity_colorless'] = [1 if x == True else 0 for x in mtg_whole_2['colorIdentity'].isna()]

mtg_whole_2.drop(columns = ['manaCost','colors'], inplace = True)
mtg_whole_2.head(2)

/home/ed/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/ed/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/ed/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

,name,names,power,toughness,text,type,types,subtypes,supertypes,rarity,colorIdentity,convertedManaCost,faceConvertedManaCost,side,manaCost_Green,manaCost_Blue,manaCost_Black,manaCost_Red,manaCost_White,manaCost_colorless,colorIdentity_Green,colorIdentity_Blue,colorIdentity_Black,colorIdentity_Red,colorIdentity_White,colorIdentity_colorless
0,Abundance,NaN,NaN,NaN,"If you would draw a card, you may instead choo...",Enchantment,Enchantment,NaN,NaN,rare,G,4.0,NaN,NaN,2,0,0,0,0,2,1,0,0,0,0,0
1,Academy Researchers,NaN,2,2,When Academy Researchers enters the battlefiel...,Creature — Human Wizard,Creature,"Human,Wizard",NaN,uncommon,U,3.0,NaN,NaN,0,2,0,0,0,1,0,1,0,0,0,0


In [8]:
# separates data by distinct card type for clustering within these.
creatures = mtg_whole_2.where(mtg_whole_2['types'].str.endswith('Creature')).dropna(how='all') # add artifact/enchanntment creatures.
spells = mtg_whole_2.where(mtg_whole_2['types'].str.endswith(('Sorcery', 'Instant'))).drop(columns = [ 'power', 'toughness']).dropna(how='all') # add trybal spells.

artifacts = mtg_whole_2.where(mtg_whole_2['types'].str.endswith('Artifact')).drop(columns = ['power', 'toughness']).dropna(how='all') # filter out artefact lands

enchantments = mtg_whole_2.where(mtg_whole_2['types'].str.endswith('Enchantment' )).drop(columns = [ 'power', 'toughness']).dropna(how='all')

planeswalkers = mtg_whole_2.where(mtg_whole_2['types'].str.endswith('Planeswalker')).drop(columns = [ 'power', 'toughness']).dropna(how='all')

In [9]:
# mtg_whole_2['types'].value_counts()
# spells['types'].value_counts()
comanders = creatures.where(creatures['type'].str.startswith('Legendary Creature')).dropna(how = 'all')

In [10]:
df['is_commander'] = mtg_whole_2.where(mtg_whole_2['types'].str.endswith('Creature')).where(creatures['type'].str.startswith('Legendary Creature')).dropna(how = 'all')

name                       869
names                       36
power                      869
toughness                  869
text                       856
type                       869
types                      869
subtypes                   869
supertypes                 869
rarity                     869
colorIdentity              863
convertedManaCost          869
faceConvertedManaCost       12
side                        36
manaCost_Green             869
manaCost_Blue              869
manaCost_Black             869
manaCost_Red               869
manaCost_White             869
manaCost_colorless         869
colorIdentity_Green        869
colorIdentity_Blue         869
colorIdentity_Black        869
colorIdentity_Red          869
colorIdentity_White        869
colorIdentity_colorless    869
dtype: int64

In [11]:
import pandas as pd
import numpy as np
import requests
from time import sleep
import json

scryfall_end = 'https://api.scryfall.com'

def get_img(url):
    sleep(3)
    get_ = requests.get(url)
    json_ = json.loads(get_.text)
    return json_['image_uris']['normal']

df = mtg_modern['scryfallId']
i = f'{scryfall_end}/cards/{df[0]}' 
get_img(i)

'https://img.scryfall.com/cards/normal/front/4/6/46184f97-d5c9-4a98-9fd9-e19057ce9b7e.jpg?1562547005'

In [117]:
df1 = pd.read_csv('mtg_modern_clean.csv')
df1.head()

,Unnamed: 0,name,power,toughness,text,types,type,subtypes,rarity,convertedManaCost,faceConvertedManaCost,side,scryfallId,manaCost_Green,manaCost_Blue,manaCost_Black,manaCost_Red,manaCost_White,manaCost_colorless,colorIdentity_Green,colorIdentity_Blue,colorIdentity_Black,colorIdentity_Red,colorIdentity_White,is_commander,uri
0,0,Abundance,NaN,NaN,"If you would draw a card, you may instead choose land or nonland and reveal cards from the top of your library until you reveal a card of the chosen kind. Put that card into your hand and put all other cards revealed this way on the bottom of your library in any order.",Enchantment,Enchantment,NaN,rare,4.0,NaN,NaN,46184f97-d5c9-4a98-9fd9-e19057ce9b7e,2,0,0,0,0,2,1,0,0,0,0,0,https://img.scryfall.com/cards/normal/front/4/6/46184f97-d5c9-4a98-9fd9-e19057ce9b7e.jpg?1562547005
1,1,Academy Researchers,2,2,"When Academy Researchers enters the battlefield, you may put an Aura card from your hand onto the battlefield attached to Academy Researchers.",Creature,Creature — Human Wizard,"Human,Wizard",uncommon,3.0,NaN,NaN,b62ca800-ec24-476e-b56f-834eef9622a9,0,2,0,0,0,1,0,1,0,0,0,0,https://img.scryfall.com/cards/normal/front/b/6/b62ca800-ec24-476e-b56f-834eef9622a9.jpg?1562553915
2,2,Adarkar Wastes,NaN,NaN,{T}: Add {C}.\n{T}: Add {W} or {U}. Adarkar Wastes deals 1 damage to you.,Land,Land,NaN,rare,0.0,NaN,NaN,f31cf014-7ac9-428b-9ce9-8ba5ebfdd252,0,0,0,0,0,0,0,1,0,0,1,0,https://img.scryfall.com/cards/normal/front/f/3/f31cf014-7ac9-428b-9ce9-8ba5ebfdd252.jpg?1562557714
3,3,Afflict,NaN,NaN,Target creature gets -1/-1 until end of turn.\nDraw a card.,Instant,Instant,NaN,common,3.0,NaN,NaN,20f18447-2853-469b-b3b3-20d6bcafa6e7,0,0,1,0,0,2,0,0,1,0,0,0,https://img.scryfall.com/cards/normal/front/2/0/20f18447-2853-469b-b3b3-20d6bcafa6e7.jpg?1562544882
4,4,Aggressive Urge,NaN,NaN,Target creature gets +1/+1 until end of turn.\nDraw a card.,Instant,Instant,NaN,common,2.0,NaN,NaN,bf92ae01-9d1e-4b41-b068-096648daadf6,1,0,0,0,0,1,1,0,0,0,0,0,https://img.scryfall.com/cards/normal/front/b/f/bf92ae01-9d1e-4b41-b068-096648daadf6.jpg?1562554491


In [11]:
import pandas as pd


#df1['uri'] = df['uri']
df1.query('uri == "0"')['name'].head(10)

6812           Werewolf Ransacker
6813           Afflicted Deserter
6828             Chalice of Death
6829              Chalice of Life
6832             Markov's Servant
6833             Chosen of Markov
6847            Withengar Unbound
6848    Elbrus, the Binding Blade
6873           Hinterland Scourge
6874            Hinterland Hermit
Name: name, dtype: object

In [118]:
import pandas as pd
data = pd.read_csv('mtg_modern_clean.csv')

In [81]:


def top_commanders(in_, frame = data): # filters commander collor identities
    frame = frame.query(f'is_commander == 1')
    uri = []
    if in_ == None: # filters colorless
        frame1 = frame.query('colorIdentity_colorless == 1')
        uri_result = list(frame1['uri'])
    else: # all others
        for i in in_:
            temp = frame[[f'colorIdentity_{i}','name','uri']]# ,'use']]
            uri += list(temp['uri'].dropna())# temp['use']))# need to get use data
        uri_frame =  pd.DataFrame(uri)  #  'use']))
        uri_frame.columns = ['uri'] #'use]
        uri_result = list(uri_frame.dropna()['uri'].unique())
    return uri_result
top_commanders(['Black'])

67bb111f5.jpg?1562552546',
 'https://img.scryfall.com/cards/normal/front/e/a/eafd2e27-01d0-4894-886e-2b8776904ab9.jpg?1562796410',
 'https://img.scryfall.com/cards/normal/front/9/7/97c2cc25-f3cd-4bfb-b9cc-53d1f42cd69a.jpg?1562791312',
 'https://img.scryfall.com/cards/normal/front/0/4/0415cc0e-979e-42cc-a56d-88d13153a7de.jpg?1570587583',
 'https://img.scryfall.com/cards/normal/front/5/b/5be8eed7-c033-42cc-bd21-4512db7af66c.jpg?1562302239',
 'https://img.scryfall.com/cards/normal/front/9/0/90cef94d-d941-4e08-afec-a116626b74fb.jpg?1562302971',
 'https://img.scryfall.com/cards/normal/front/5/e/5e058ff8-043c-498b-8310-0ca45466ac27.jpg?1562302266',
 'https://img.scryfall.com/cards/normal/front/1/a/1a1d95b9-aa18-41e2-b972-93fda25e0b11.jpg?1562300734',
 'https://img.scryfall.com/cards/normal/front/d/1/d19fdc00-21eb-48dc-966a-6b634dc5a2c4.jpg?1562304374',
 'https://img.scryfall.com/cards/normal/front/e/e/ee338221-ead9-4b89-8b0c-12745c4ca13d.jpg?1562304880',
 'https://img.scryfall.com/cards/norm

In [29]:
pd.set_option('max_colwidth',400)
data.query('is_commander == "1"').head()[['name', 'uri']]

,name,uri
7,Ambassador Laquatus,https://img.scryfall.com/cards/normal/front/6/f/6fdd01bd-ab41-4005-8807-46db0cfc4da4.jpg?1562549580
16,Arcanis the Omnipotent,https://img.scryfall.com/cards/normal/front/b/b/bba2cd91-84a7-4d05-9a0c-55440491be1e.jpg?1562554259
17,Ascendant Evincar,https://img.scryfall.com/cards/normal/front/3/a/3a67d7ae-3117-433f-ad82-44a4f015fecf.jpg?1562546342
47,"Cho-Manno, Revolutionary",https://img.scryfall.com/cards/normal/front/d/a/da6b8e11-b799-4235-bd8f-c4b55946c769.jpg?1562556139
153,"Kamahl, Pit Fighter",https://img.scryfall.com/cards/normal/front/9/8/983874b1-3179-4ac6-a4f3-efa133331c6f.jpg?1562552072


In [30]:
data.query('name=="Arcanis the Omnipotent"')['scryfallId']

16    0ed53483-4bec-491b-9ae6-afc6faf122a2
Name: scryfallId, dtype: object

In [36]:
def image_uri():
    temp = pd.read_csv('Images/img_uri.csv')
    temp.columns = ['index', 'uri']
    temp.set_index('index', inplace =True)
    return temp['uri']
# image_uri().head(17)

In [ ]:
d_in_ = '10001'

temp.query(f"color_identity == {d_in_}")
temp = frame[['name','uri']]# ,'use']]
uri += list(temp['uri'].dropna())# temp['use']))# need to get use data
uri_frame =  pd.DataFrame(uri)  #  'use']))
uri_frame.columns = ['uri'] #'use]
uri_result = list(uri_frame.dropna().query('uri != "0"')['uri'].unique())
return uri_result #.sortvalues(by =['use'], ascending = False) # as above

In [120]:
def top_commanders(in_, frame = data): # filters commander collor identities
    colors =  ['Black','White','Red','Blue','Green']
    frame = frame.query('is_commander == 1')
    if in_ == [False,False,False,False,False]:
        color = ['colorless']
    else:
        color = [b for a, b in zip(in_, colors) if a]
    for i in color:
        frame = frame.query(f'colorIdentity_{i} == 1')
    return list(frame['name'])
top_commanders([False,False,False,False,False])

['Ulamog, the Ceaseless Hunger',
 'Emrakul, the Promised End',
 'Emrakul, the Aeons Torn',
 'Kozilek, Butcher of Truth',
 'Ulamog, the Infinite Gyre',
 'Kozilek, the Great Distortion']

In [99]:
data.query('color_identity == [0, 0, 0, 0, 0]')

,Unnamed: 0,name,power,toughness,text,types,type,subtypes,rarity,convertedManaCost,faceConvertedManaCost,side,scryfallId,manaCost_Green,manaCost_Blue,manaCost_Black,manaCost_Red,manaCost_White,manaCost_colorless,colorIdentity_Green,colorIdentity_Blue,colorIdentity_Black,colorIdentity_Red,colorIdentity_White,color_identity,is_commander,uri
